#### Active learning NNGP experiment
multiple params

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt

from model.mlp import MLP
from dataloader.rosen import RosenData
from uncertainty_estimator.nngp import NNGP
from uncertainty_estimator.mcdue import MCDUE
from uncertainty_estimator.random_estimator import RandomEstimator
from sample_selector.eager import EagerSampleSelector
from oracle.identity import IdentityOracle
from al_trainer import ALTrainer

import random

/home/beardysome/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/beardysome/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
/home/beardysome/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/beardysome/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/beardysome/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 

In [3]:
config = {
    'random_seed': 4623457,
    'n_dim': 10,
    'n_train': 200,
    'n_test': 200,
    'n_pool': 1000,
    'layers': [128, 64, 32],
    'update_sample_size': 100,
    'al_iterations': 10
}

if config['random_seed'] is not None:
    np.random.seed(config['random_seed'])
    random.seed(config['random_seed'])

In [4]:
def print_shapes(note, *sets):
    print(note)
    for x, y in sets:
        print("shapes:", x.shape, y.shape)

# load data
X_train, y_train, _, _, X_test, y_test, X_pool, y_pool = RosenData(
    config['n_train'], 200, config['n_test'], config['n_pool'], config['n_dim']
).dataset(use_cache=True)

In [5]:
# Init neural network & tf session
tf.reset_default_graph()
if config['random_seed'] is not None:
    tf.set_random_seed(config['random_seed'])

model = MLP(
    ndim = config['n_dim'],
    random_state = config['random_seed'],
    layers = config['layers']
)

try:
    sess.close()
except:
    pass


init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

model.set_session(sess)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [6]:
estimator = NNGP(model)  # to estimate uncertainties
estimator.set_session(sess)
oracle = IdentityOracle(y_pool)  # generate y for X from pool
sampler = EagerSampleSelector(oracle) # sample X and y from pool by uncertainty estimations

# estimator.estimate(X_train, y_train, X_pool)[:40]

In [8]:
trainer = ALTrainer(model, estimator, sampler, oracle, config['al_iterations'], config['update_sample_size'])
rmses = trainer.train(X_train, y_train, X_test, y_test, X_test, y_test, X_pool)


[100]  RMSE train:198.612  test:189.905  val:189.905  patience:3
[200]  RMSE train:194.201  test:185.435  val:185.435  patience:3
[300]  RMSE train:182.417  test:173.510  val:173.510  patience:3
[400]  RMSE train:158.313  test:149.157  val:149.157  patience:3
[500]  RMSE train:121.634  test:112.368  val:112.368  patience:3
[600]  RMSE train:85.051  test:77.082  val:77.082  patience:3
[700]  RMSE train:68.410  test:64.145  val:64.145  patience:3
[800]  RMSE train:66.035  test:64.088  val:64.088  patience:3
[900]  RMSE train:65.857  test:64.496  val:64.496  patience:2
[1000]  RMSE train:65.783  test:64.537  val:64.537  patience:1
[1100]  RMSE train:65.708  test:64.499  val:64.499  patience:0
No patience left at epoch 1100. Early stopping.
Uncertainties [4171.68554688 4098.71337891    0.         1447.96569824    0.
 1910.39746094 3555.6484375  4405.49804688    0.            0.
 2836.44824219 4999.81689453 5808.04150391 2468.97900391  861.36071777
 5367.02099609 4705.95410156 6512.859375  

[500]  RMSE train:16.407  test:26.490  val:26.490  patience:3
[600]  RMSE train:15.793  test:26.303  val:26.303  patience:3
[700]  RMSE train:15.247  test:26.156  val:26.156  patience:3
[800]  RMSE train:14.768  test:26.091  val:26.091  patience:3
[900]  RMSE train:14.340  test:26.009  val:26.009  patience:3
[1000]  RMSE train:13.956  test:25.931  val:25.931  patience:3
[1100]  RMSE train:13.603  test:25.831  val:25.831  patience:3
[1200]  RMSE train:13.257  test:25.701  val:25.701  patience:3
[1300]  RMSE train:12.944  test:25.581  val:25.581  patience:3
[1400]  RMSE train:12.656  test:25.454  val:25.454  patience:3
[1500]  RMSE train:12.373  test:25.380  val:25.380  patience:3
[1600]  RMSE train:12.107  test:25.302  val:25.302  patience:3
[1700]  RMSE train:11.849  test:25.263  val:25.263  patience:3
[1800]  RMSE train:11.590  test:25.271  val:25.271  patience:2
[1900]  RMSE train:11.337  test:25.234  val:25.234  patience:3
[2000]  RMSE train:11.110  test:25.206  val:25.206  patience

[100]  RMSE train:11.596  test:22.141  val:22.141  patience:3
[200]  RMSE train:11.198  test:21.952  val:21.952  patience:3
[300]  RMSE train:10.917  test:21.833  val:21.833  patience:3
[400]  RMSE train:10.684  test:21.783  val:21.783  patience:3
[500]  RMSE train:10.480  test:21.753  val:21.753  patience:3
[600]  RMSE train:10.305  test:21.697  val:21.697  patience:3
[700]  RMSE train:10.127  test:21.626  val:21.626  patience:3
[800]  RMSE train:9.960  test:21.582  val:21.582  patience:3
[900]  RMSE train:9.799  test:21.534  val:21.534  patience:3
[1000]  RMSE train:9.641  test:21.513  val:21.513  patience:3
[1100]  RMSE train:9.478  test:21.507  val:21.507  patience:3
[1200]  RMSE train:9.323  test:21.488  val:21.488  patience:3
[1300]  RMSE train:9.180  test:21.478  val:21.478  patience:3
[1400]  RMSE train:9.042  test:21.500  val:21.500  patience:2
[1500]  RMSE train:8.912  test:21.493  val:21.493  patience:1
[1600]  RMSE train:8.781  test:21.446  val:21.446  patience:3
[1700]  RM

[2500]  RMSE train:5.028  test:17.710  val:17.710  patience:3
[2600]  RMSE train:4.989  test:17.688  val:17.688  patience:3
[2700]  RMSE train:4.949  test:17.684  val:17.684  patience:3
[2800]  RMSE train:4.912  test:17.681  val:17.681  patience:3
[2900]  RMSE train:4.877  test:17.684  val:17.684  patience:2
[3000]  RMSE train:4.842  test:17.648  val:17.648  patience:3
[3100]  RMSE train:4.805  test:17.629  val:17.629  patience:3
[3200]  RMSE train:4.769  test:17.627  val:17.627  patience:3
[3300]  RMSE train:4.732  test:17.602  val:17.602  patience:3
[3400]  RMSE train:4.688  test:17.615  val:17.615  patience:2
[3500]  RMSE train:4.646  test:17.595  val:17.595  patience:3
[3600]  RMSE train:4.605  test:17.569  val:17.569  patience:3
[3700]  RMSE train:4.565  test:17.564  val:17.564  patience:3
[3800]  RMSE train:4.526  test:17.512  val:17.512  patience:3
[3900]  RMSE train:4.485  test:17.507  val:17.507  patience:3
[4000]  RMSE train:4.445  test:17.476  val:17.476  patience:3
[4100]  

[5700]  RMSE train:3.634  test:13.663  val:13.663  patience:1
[5800]  RMSE train:3.620  test:13.639  val:13.639  patience:0
No patience left at epoch 5800. Early stopping.
Iteration 10 RMSE 13.638887670142065


In [ ]:
print(rmses)
plt.plot(rmses, label='NNGP', marker='.')
plt.title('RMS Error by active learning iterations')
plt.legend()

In [ ]:
# Show some predictions
print(model.predict(data = X_test[:3]))
print(y_test[:3])

In [ ]:
# show some uncertainties
estimator.estimate(X_train, y_train, X_pool)[:3]